# MNIST CNN

### The images are 28x28 pixels in dimension and is in grayscale

#### Importing the basic libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

W0509 01:42:39.900890 20052 deprecation.py:323] From <ipython-input-1-4955fd1ce053>:8: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0509 01:42:39.902886 20052 deprecation.py:323] From C:\Users\obero\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0509 01:42:39.903857 20052 deprecation.py:323] From C:\Users\obero\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0509 01:42:40.102327 20052 deprecation.py:323] From C:\Users\obero\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0509 01:42:40.144214 20052 deprecation.py:323] From C:\Users\obero\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Initializing the weights and bias

In [2]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [3]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

#### Creating the convolution and pooling functions

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [5]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

#### Creating convolutional layer and normal layer

In [6]:
# we'll return an actual convolutional layer here that uses an ReLu activation.
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [7]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

#### Creating x, y_true and hold_probability placeholders(For Dropout)

In [13]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])
hold_prob = tf.placeholder(tf.float32)

#### Reshaping the images and stacking them vertically

In [14]:
x_image = tf.reshape(x,[-1,28,28,1])

#### Layer 1

In [15]:
# Using a 6 by 6 filter here
# 32 Output
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

#### Layer 2

In [16]:
# Using a 6 by 6 filter here
# 64 Output 
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [17]:
# 7 by 7 image because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

#### Dropout

In [18]:
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

W0509 01:46:49.767288 20052 deprecation.py:506] From <ipython-input-18-e648e17ffd84>:1: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
y_pred = normal_full_layer(full_one_dropout,10)

#### Cross Entropy Loss Function

In [20]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

W0509 01:47:11.699894 20052 deprecation.py:323] From <ipython-input-20-e0c07892321c>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



#### Adam Train Optimizer

In [21]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

#### Initializing init

In [22]:
init = tf.global_variables_initializer()

#### Running the CNN 

In [23]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1425


Currently on step 100
Accuracy is:
0.8589


Currently on step 200
Accuracy is:
0.9075


Currently on step 300
Accuracy is:
0.9257


Currently on step 400
Accuracy is:
0.9381


Currently on step 500
Accuracy is:
0.9488


Currently on step 600
Accuracy is:
0.9522


Currently on step 700
Accuracy is:
0.9527


Currently on step 800
Accuracy is:
0.9594


Currently on step 900
Accuracy is:
0.9551


Currently on step 1000
Accuracy is:
0.9634


Currently on step 1100
Accuracy is:
0.9655


Currently on step 1200
Accuracy is:
0.9684


Currently on step 1300
Accuracy is:
0.969


Currently on step 1400
Accuracy is:
0.971


Currently on step 1500
Accuracy is:
0.9714


Currently on step 1600
Accuracy is:
0.9737


Currently on step 1700
Accuracy is:
0.9716


Currently on step 1800
Accuracy is:
0.975


Currently on step 1900
Accuracy is:
0.9758


Currently on step 2000
Accuracy is:
0.9776


Currently on step 2100
Accuracy is:
0.9772


Currently on step 2200
Ac